In [8]:
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dot, Add
from keras.layers import  Dropout, BatchNormalization, Activation
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

In [2]:
# load in the data
df = pd.read_csv('C:/Users/Jeferson Santana/OneDrive/Documentos/recommender_course/data/edited_rating.csv')

In [3]:
N = df.userId.max() + 1 # number of users
M = df.movie_idx.max() + 1 # number of movies

In [4]:
# split into train and test
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

In [6]:
# initialize variables
K = 10 # latent dimensionality
mu = df_train.rating.mean()
epochs = 25
reg = 0.

In [7]:
# keras model
u = Input(shape=(1,))
m = Input(shape=(1,))
u_embedding = Embedding(N, K)(u)
m_embedding = Embedding(M, K)(m)

In [9]:
# main branch
u_bias = Embedding(N, 1)(u)
m_bias = Embedding(M, 1)(m)
x = Dot(axes=2)([u_embedding, m_embedding])
x = Add()([x, u_bias, m_bias])
x = Flatten()(x)

In [10]:
# side branch
u_embedding = Flatten()(u_embedding)
m_embedding = Flatten()(m_embedding)
y = Concatenate()([u_embedding, m_embedding])
y = Dense(400)(y)
y = Activation('elu')(y)
y = Dense(1)(y)

In [11]:
# merge
x = Add()([x, y])

In [ ]:
model = Model(inputs=[u ,m], outputs=x)
model.compile(
                loss='mse',
                optimizer=SGD(learning_rate=0.01, momentum=0.9),
                metrics=['mse']
             )

r = model.fit(
                x=[df_train.userId.values, df_train.movie_idx.values],
                y=df_train.rating.values - mu,
                epochs=epochs,
                batch_size=128,
                validation_data=(
                                    [df_test.userId.values, df_test.movie_idx.values],
                                    df_test.rating.values - mu
                                )
              )


In [ ]:
# plot losses
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label="test loss")
plt.legend()
plt.show()

# plot mse
plt.plot(r.history['mean_squared_error'], label="train mse")
plt.plot(r.history['val_mean_squared_error'], label="test mse")
plt.legend()
plt.show()